# Setting

In [1]:
# import libraries
import mne, os, pickle, glob, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as stats
from mne.stats import (spatio_temporal_cluster_test, f_mway_rm, f_threshold_mway_rm)

In [2]:
# directories
mri_dir  = os.path.realpath('../Data/mri')     # mri directory
meg_dir  = os.path.realpath('../Data/meg-exp') # meg directory
log_dir  = os.path.join(meg_dir, 'log')        # log directory
stc_dir  = os.path.join(meg_dir, 'stc')        # stc directory
mod_dir  = os.path.join(meg_dir, 'mod')        # GLM model directory
res_dir  = os.path.join(meg_dir, 'res')        # results directory

# subject list
subjects = [
    'P049','P050','P054','P055','P056',
    'P057','P058','P059','P060','P061',
    'P062','P063','P064','P065','P068',
    'P069','P070','P071'
    ]

# define anatomical mask
The anatomical mask is based on the meta-analysis in [Leminen et al. (2019) *Cortex*](https://doi.org/10.1016/j.cortex.2018.08.016) and includes the following brain regions from the Desikan-Killiany Atlas:
- parsopercularis
- parsorbitalis
- parstriangularis
- frontalpole
- lateralorbitofrontal
- medialorbitofrontal
- superiortemporal
- middletemporal
- bankssts
- transversetemporal
- insula

In [3]:
# read labels
labels = mne.read_labels_from_annot('fsaverage', 'aparc', 'both', subjects_dir=mri_dir)

# define ROIs
roi_list = ['parsopercularis', 'parsorbitalis', 'parstriangularis',
            'frontalpole', 'lateralorbitofrontal', 'medialorbitofrontal',
            'superiortemporal', 'middletemporal', 'bankssts', 'transversetemporal', 'insula']

# loop over ROI list
for roi_count, roi_name in enumerate(roi_list):
    
    # look for ROI info from the annotation labels
    roi_lh_tmp = [label for label in labels if label.name == f'{roi_name}-lh'][0]
    roi_rh_tmp = [label for label in labels if label.name == f'{roi_name}-rh'][0]
    
    # combine ROIs
    if roi_count == 0:
        roi_lh = roi_lh_tmp
        roi_rh = roi_rh_tmp
    else:
        roi_lh += roi_lh_tmp
        roi_rh += roi_rh_tmp

# set up vertex indices
n_hemisources = 2562  # number of vertices in each hemisphere
hemi_idx = np.arange(0, n_hemisources, 1)  # create an array

# look for ROI vertex indices in the LH, RH, and both himispheres
roi_idx_lh = roi_lh.get_vertices_used(vertices=hemi_idx)
roi_idx_rh = roi_rh.get_vertices_used(vertices=hemi_idx)
roi_idx_bh = np.concatenate((roi_idx_lh, roi_idx_rh+n_hemisources))

# look for ROI vertex indices NOT in the LH, RH, and both himispheres
diff_idx_lh = np.setdiff1d(hemi_idx, roi_idx_lh)
diff_idx_rh = np.setdiff1d(hemi_idx, roi_idx_rh)
diff_idx_bh = np.concatenate((diff_idx_lh, diff_idx_rh+n_hemisources))

Reading labels from parcellation...
   read 35 labels from /Users/cl5564/Library/CloudStorage/Dropbox/OSF/TagalogViolation/Data/mri/fsaverage/label/lh.aparc.annot
   read 34 labels from /Users/cl5564/Library/CloudStorage/Dropbox/OSF/TagalogViolation/Data/mri/fsaverage/label/rh.aparc.annot


# load beta map
- Model: 'constant', 'ArgStrViolNAG', 'ArgStrViolNA', 'CatViolNAG', 'CatViolNA', 'GrammNAG', 'GrammNA', 'Filler', 'LogBaseFrequency', 'WordLength', 'RespACC', 'trial order'

In [4]:
# define event codes
event_id  = dict(ArgStrViolNAG = 10, ArgStrViolNA = 20, 
                 CatViolNAG    = 30, CatViolNA    = 40, 
                 GrammNAG      = 50, GrammNA      = 60, 
                 Filler        = 70)
cond_code = list(event_id.values())  # condition code
cond_name = list(event_id.keys())    # condition name

# define regressors (12 regressors)
# constant + 7 conditions + LogBaseFrequency + WordLength + accuracy + trial order
col_labels = cond_name.copy()
col_labels.insert(0, 'constant')
for item in ['LogBaseFreq', 'Length', 'RespACC', 'trial order']: col_labels.append(item)

# define variables
epoch_tmin = -0.1  # epoch onset
epoch_tmax = 0.6   # epoch offset
times      = np.arange(epoch_tmin*1000, epoch_tmax*1000+1, 1)
n_subj     = len(subjects)    # number of subjects
n_cond     = len(cond_code)   # number of conditions
n_reg      = len(col_labels)  # number of regressors
n_times    = len(times)       # numner of time points
n_sources  = n_hemisources*2  # number of sources

# create an empty matrix for storing beta map
data_mtx  = np.empty((n_subj, n_sources, n_times, n_reg))

# load beta map
for s, subj in enumerate(subjects):
    print(subj, end=' ')
    fname    = os.path.join(mod_dir, '%s_reg%s_b-map.npy') %(subj, n_reg)
    data_val = np.load(fname)
    data_mtx[s,:,:,:] = data_val

data_mtx = data_mtx[:,:,:,1:n_cond+1] # keep condition regressors
print();print(data_mtx.shape)

P049 P050 P054 P055 P056 P057 P058 P059 P060 P061 P062 P063 P064 P065 P068 P069 P070 P071 
(18, 5124, 701, 7)


## define stats function of rmANOVA

In [5]:
def stat_fun(*args):
    # get f-values only.
    return f_mway_rm(np.swapaxes(args, 1, 0), factor_levels=factor_levels,
                     effects=effects, return_pvals=return_pvals)[0]


## licensing stage

In [6]:
# cell arrangement: A1B1 A1B2 A2B1 A2B2
# 'A': main effect of A (violation type)
# 'B': main effect of B (prefix type)
# 'A:B': interaction effect

factor_levels = [2, 2]
effects       = 'A:B'
return_pvals  = False # Tell the ANOVA not to compute p-values which we don't need for clustering

In [7]:
# define time of interest
toi_min = 150
toi_max = 300
toi     = np.arange(toi_min, toi_max+1, 1)
toi_min_idx = np.squeeze(np.where(times==toi_min))
toi_max_idx = np.squeeze(np.where(times==toi_max))
toi_idx     = np.arange(toi_min_idx,toi_max_idx+1)
toi_ntimes  = toi_idx.size

# prep for rmANOVA
n_cell  = np.prod(factor_levels) # number of cells
data_bh = [] # create an empty list for storing data of both hemispheres
data_lh = [] # create an empty list for storing data of LH
data_rh = [] # create an empty list for storing data of RH

# retrieve data for the conditions of interest
data    = data_mtx[:,:,:,1:n_cell+1]

for i in range(n_cell):
    
    data_temp = data[:,:,toi_idx][:,:,:,i]         # select data based on time of interest
    data_temp = np.transpose(data_temp, [0, 2, 1]) # transpose the matrix into: subjects x times x sources 
    data_bh.append(data_temp)
    data_lh.append(data_temp[:,:,:n_hemisources])
    data_rh.append(data_temp[:,:,n_hemisources:])

print(np.shape(data_bh))
print(np.shape(data_lh))
print(np.shape(data_rh))

(4, 18, 151, 5124)
(4, 18, 151, 2562)
(4, 18, 151, 2562)


In [8]:
# define parameters
hemi            = 'lh'
X               = eval('data_%s'%hemi)
spatial_exclude = eval('diff_idx_%s'%hemi)
tail            = 1 # F test: use the upper tail (see also: https://stats.stackexchange.com/a/73993)
p_thresh        = 0.05
n_permutations  = 10000
df              = n_subj-1
f_thresh        = f_threshold_mway_rm(n_subj, factor_levels, effects, p_thresh)

# read source space & compute adjacency
src_fname = os.path.join(mri_dir, 'fsaverage', 'bem', 'fsaverage-ico-4-src.fif')
src = mne.read_source_spaces(src_fname)
if hemi == 'lh':
    adjacency = mne.spatial_src_adjacency(src[:1]) 
elif hemi == 'rh':
    adjacency = mne.spatial_src_adjacency(src[1:])
elif hemi == 'bh':
    adjacency = mne.spatial_src_adjacency(src)

# permutation test with TFCE
print('permutation test...')
threshold_tfce = dict(start=np.ceil(f_thresh), step=0.1)
f_tfce, clusters, p_tfce, H0 = clu = \
    spatio_temporal_cluster_test(X,
                                 tail            = tail,
                                 threshold       = threshold_tfce,
                                 stat_fun        = stat_fun,
                                 n_permutations  = n_permutations,
                                 adjacency       = adjacency,
                                 spatial_exclude = spatial_exclude,
                                 n_jobs          = 10,
                                 seed            = 1119,
                                 buffer_size     = None,
                                 verbose         = True)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 2562
permutation test...
stat_fun(H1): min=0.000000 max=45.464814
Running initial clustering …
Using 405 thresholds from 5.00 to 45.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 386862 clusters


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

In [9]:
p_thresh = 0.05
pval     = np.reshape(p_tfce, [toi_ntimes, n_hemisources])
pval_cmp = pval <= p_thresh
    
vidx = np.unique(np.where(pval_cmp)[1])
tidx = np.unique(np.where(pval_cmp)[0])
clu_tmin  = toi[tidx[0]]
clu_tmax  = toi[tidx[-1]]
clu_times = np.arange(clu_tmin, clu_tmax+1)

if clu_times.size == tidx.size:
    print('time range: %s-%sms' %(clu_tmin, clu_tmax))
    print('LH vertices: %s' %(vidx.size))
else:
    print('time range is not continuous...')

time range: 175-217ms
LH vertices: 126


In [10]:
# save data
if effects == 'A':
    effect_name = 'mainViol'
elif effects == 'B':
    effect_name = 'mainPrefix'
elif effects == 'A:B':
    effect_name = 'Interaction'

current_date  = datetime.date.today()
contrast_name = 'reg%s_%s-%s' %(n_reg, effect_name, hemi)
pickle_fname  = os.path.join(res_dir, 'res_%s_%s-%s_%ssubjs_tfce_%s.pickled') % (contrast_name, toi_min, toi_max, n_subj, current_date)
open_file     = open(pickle_fname, "wb")
pickle.dump(clu, open_file)
open_file.close()

## composition stage

In [11]:
# cell arrangement: A1B1 A1B2 A2B1 A2B2
# 'A': main effect of A (violation type)
# 'B': main effect of B (prefix type)
# 'A:B': interaction effect

factor_levels = [2, 2]
effects       = 'A'
return_pvals  = False # Tell the ANOVA not to compute p-values which we don't need for clustering

In [12]:
# define time of interest
toi_min = 300
toi_max = 450
toi     = np.arange(toi_min, toi_max+1, 1)
toi_min_idx = np.squeeze(np.where(times==toi_min))
toi_max_idx = np.squeeze(np.where(times==toi_max))
toi_idx     = np.arange(toi_min_idx,toi_max_idx+1)
toi_ntimes  = toi_idx.size

# prep for rmANOVA
n_cell  = np.prod(factor_levels) # number of cells
data_bh = [] # create an empty list for storing data of both hemispheres
data_lh = [] # create an empty list for storing data of LH
data_rh = [] # create an empty list for storing data of RH

# retrieve data for the conditions of interest
data    = data_mtx[:,:,:,1:n_cell+1]

for i in range(n_cell):
    
    data_temp = data[:,:,toi_idx][:,:,:,i]         # select data based on time of interest
    data_temp = np.transpose(data_temp, [0, 2, 1]) # transpose the matrix into: subjects x times x sources 
    data_bh.append(data_temp)
    data_lh.append(data_temp[:,:,:n_hemisources])
    data_rh.append(data_temp[:,:,n_hemisources:])

print(np.shape(data_bh))
print(np.shape(data_lh))
print(np.shape(data_rh))

(4, 18, 151, 5124)
(4, 18, 151, 2562)
(4, 18, 151, 2562)


In [13]:
# define parameters
hemi            = 'lh'
X               = eval('data_%s'%hemi)
spatial_exclude = eval('diff_idx_%s'%hemi)
tail            = 1 # F test: use the upper tail (see also: https://stats.stackexchange.com/a/73993)
p_thresh        = 0.05
n_permutations  = 10000
df              = n_subj-1
f_thresh        = f_threshold_mway_rm(n_subj, factor_levels, effects, p_thresh)

# read source space & compute adjacency
src_fname = os.path.join(mri_dir, 'fsaverage', 'bem', 'fsaverage-ico-4-src.fif')
src = mne.read_source_spaces(src_fname)
if hemi == 'lh':
    adjacency = mne.spatial_src_adjacency(src[:1]) 
elif hemi == 'rh':
    adjacency = mne.spatial_src_adjacency(src[1:])
elif hemi == 'bh':
    adjacency = mne.spatial_src_adjacency(src)

# permutation test with TFCE
print('permutation test...')
threshold_tfce = dict(start=np.ceil(f_thresh), step=0.1)
f_tfce, clusters, p_tfce, H0 = clu = \
    spatio_temporal_cluster_test(X,
                                 tail            = tail,
                                 threshold       = threshold_tfce,
                                 stat_fun        = stat_fun,
                                 n_permutations  = n_permutations,
                                 adjacency       = adjacency,
                                 spatial_exclude = spatial_exclude,
                                 n_jobs          = 10,
                                 seed            = 1119,
                                 buffer_size     = None,
                                 verbose         = True)

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
-- number of adjacent vertices : 2562
permutation test...
stat_fun(H1): min=0.000000 max=45.049390
Running initial clustering …
Using 401 thresholds from 5.00 to 45.00 for TFCE computation (h_power=2.00, e_power=0.50)
Found 386862 clusters


  0%|          | Permuting : 0/9999 [00:00<?,       ?it/s]

In [14]:
p_thresh = 0.05
pval     = np.reshape(p_tfce, [toi_ntimes, n_hemisources])
pval_cmp = pval <= p_thresh
    
vidx = np.unique(np.where(pval_cmp)[1])
tidx = np.unique(np.where(pval_cmp)[0])
clu_tmin  = toi[tidx[0]]
clu_tmax  = toi[tidx[-1]]
clu_times = np.arange(clu_tmin, clu_tmax+1)

if clu_times.size == tidx.size:
    print('time range: %s-%sms' %(clu_tmin, clu_tmax))
    print('LH vertices: %s' %(vidx.size))
else:
    print('time range is not continuous...')

time range: 361-398ms
LH vertices: 95


In [15]:
# save data
if effects == 'A':
    effect_name = 'mainViol'
elif effects == 'B':
    effect_name = 'mainPrefix'
elif effects == 'A:B':
    effect_name = 'Interaction'

current_date  = datetime.date.today()
contrast_name = 'reg%s_%s-%s' %(n_reg, effect_name, hemi)
pickle_fname  = os.path.join(res_dir, 'res_%s_%s-%s_%ssubjs_tfce_%s.pickled') % (contrast_name, toi_min, toi_max, n_subj, current_date)
open_file     = open(pickle_fname, "wb")
pickle.dump(clu, open_file)
open_file.close()